In [1]:
# Importation des librairies.
%autosave 0
import pandas as pd
import joblib
import nltk
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from scipy.sparse import hstack

# Téléchargement de la liste de stop words en français.
nltk.download('stopwords')

# Importation de la liste de stop words en français.
from nltk.corpus import stopwords
stop_words_french = set(stopwords.words('french'))

Autosave disabled


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Windows\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Lecture du fichier.
df = pd.read_csv("df2.csv")
df.head()

,Titre,Commentaire,Date_experience,Date_publication,Reponse,Date_reponse,Pays,Verification,Nombre_avis_publie,Note,...,Titre_3grame_pas_recu_pneus,Titre_3grame_livraison_trop_long,Titre_3grame_livraison_trop_longue,Titre_3grame_rapport_qualite_prix,Titre_3grame_livraison_rapide_conforme,Titre_3grame_livraison_peu_longue,Titre_3grame_livraison_rapide_prix,polarity_Commentaire,polarity_Titres,Saison_experience
0,Je commande mes pneus de motos et de voitures,Je commande mes pneus de motos et de voitures ...,2023-07-23,2023-08-05,NaN,NaN,GR,1,1,5,...,0,0,0,0,0,0,0,0.271000,0.00,3
1,"Ne cherchez plus vos pneus, Allopneus le peut","BonjourLe site est genial, tres bien explique ...",2023-07-13,2023-08-04,NaN,NaN,FR,1,6,5,...,0,0,0,0,0,0,0,0.400000,0.15,3
2,Tout etait parfait,"Tout etait parfait, timing parfait dans les da...",2023-07-16,2023-08-04,NaN,NaN,FR,1,3,5,...,0,0,0,0,0,0,0,0.391667,0.51,3
3,Pneus d'un tres bon rapport qualite_,Pneus d'un tres bon rapport qualite prix. Les ...,2023-07-24,2023-08-05,NaN,NaN,FR,1,4,5,...,0,0,0,0,0,0,0,0.607500,1.00,3
4,"Bon suivi de la commande,les questions_","Bon suivi de la commande,les questions sont im...",2023-07-23,2023-08-04,NaN,NaN,FR,1,1,4,...,0,0,0,0,0,0,0,0.133333,0.70,3


In [3]:
# Création de la colonne "Sentiment" à partir de la colonne "Note".
df['Sentiment'] = df['Note'].apply(lambda x: 'Négatif' if x in [1, 2] else 'Positif')

# Convertir les valeurs de la colonne "Sentiment" en valeurs numériques.
sentiment_encoder = LabelEncoder()
df['Sentiment'] = sentiment_encoder.fit_transform(df['Sentiment'])

In [4]:
# Suppression des caractères spéciaux, chiffres, valeurs manquantes et texte en minuscules.
df['Titre'] = df['Titre'].str.replace('[^a-zA-Z\s]', '').str.lower()
df['Titre'] = df['Titre'].fillna('') 
df['Commentaire'] = df['Commentaire'].str.replace('[^a-zA-Z\s]', '').str.lower()
df['Commentaire'] = df['Commentaire'].fillna('') 
df.head()

,Titre,Commentaire,Date_experience,Date_publication,Reponse,Date_reponse,Pays,Verification,Nombre_avis_publie,Note,...,Titre_3grame_livraison_trop_long,Titre_3grame_livraison_trop_longue,Titre_3grame_rapport_qualite_prix,Titre_3grame_livraison_rapide_conforme,Titre_3grame_livraison_peu_longue,Titre_3grame_livraison_rapide_prix,polarity_Commentaire,polarity_Titres,Saison_experience,Sentiment
0,je commande mes pneus de motos et de voitures,je commande mes pneus de motos et de voitures ...,2023-07-23,2023-08-05,NaN,NaN,GR,1,1,5,...,0,0,0,0,0,0,0.271000,0.00,3,1
1,"ne cherchez plus vos pneus, allopneus le peut","bonjourle site est genial, tres bien explique ...",2023-07-13,2023-08-04,NaN,NaN,FR,1,6,5,...,0,0,0,0,0,0,0.400000,0.15,3,1
2,tout etait parfait,"tout etait parfait, timing parfait dans les da...",2023-07-16,2023-08-04,NaN,NaN,FR,1,3,5,...,0,0,0,0,0,0,0.391667,0.51,3,1
3,pneus d'un tres bon rapport qualite_,pneus d'un tres bon rapport qualite prix. les ...,2023-07-24,2023-08-05,NaN,NaN,FR,1,4,5,...,0,0,0,0,0,0,0.607500,1.00,3,1
4,"bon suivi de la commande,les questions_","bon suivi de la commande,les questions sont im...",2023-07-23,2023-08-04,NaN,NaN,FR,1,1,4,...,0,0,0,0,0,0,0.133333,0.70,3,1


In [5]:
# Sélection des colonnes à exclure.
colonnes_exclues = ['Titre', 'Date_experience', 'Date_publication', 'Reponse', 'Date_reponse', 'Pays', 'Verification', 'Note', 'Entreprise', 'Sentiment']

# Sélection des colonnes à inclure dans X.
colonnes_incluses = [colonne for colonne in df.columns if colonne not in colonnes_exclues]

# Séparation des données en X (caractéristiques) et y (variable cible).
X = df[colonnes_incluses]
y = df['Sentiment']

In [6]:
# Division des données en ensembles d'entraînement et de test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Extraire la colonne 'Commentaire' de X_train et X_test.
X_train_commentaire = X_train['Commentaire']
X_test_commentaire = X_test['Commentaire']

# Appliquer une vectorisation de texte via CountVectorizer avec stop words en français.
vectorizer = CountVectorizer(stop_words=list(stop_words_french))
X_train_commentaire_vectorized = vectorizer.fit_transform(X_train_commentaire)
X_test_commentaire_vectorized = vectorizer.transform(X_test_commentaire)

In [8]:
# Convertir les données vectorisées en matrices creuses de type float64.
X_train_commentaire_vectorized = X_train_commentaire_vectorized.astype('float64')
X_test_commentaire_vectorized = X_test_commentaire_vectorized.astype('float64')

In [9]:
# Concaténer les matrices creuses des commentaires avec les autres colonnes.
X_train_concat = hstack([X_train_commentaire_vectorized] + [X_train.drop('Commentaire', axis=1).values.astype('float64')])
X_test_concat = hstack([X_test_commentaire_vectorized] + [X_test.drop('Commentaire', axis=1).values.astype('float64')])

#### Gradient Boosting

In [10]:
# Créer un classificateur clf en utilisant la méthode Gradient Boosting.
gb_clf = GradientBoostingClassifier()

# Entraîner le classificateur sur l'ensemble d'entraînement.
gb_clf.fit(X_train_concat, y_train)
joblib.dump(gb_clf, 'gb_clf.joblib')

# Faire des prédictions sur l'ensemble de test.
predictions = gb_clf.predict(X_test_concat)

In [11]:
# Affichage du rapport de classification du modèle Gradient Boosting.
print("Rapport de classification :\n", classification_report(y_test, predictions))

# Calcul et affichage de la matrice de confusion.
confusion_matrix = pd.crosstab(y_test, predictions, rownames=['Classe réelle'], colnames=['Classe prédite'])
confusion_matrix

Rapport de classification :
               precision    recall  f1-score   support

           0       0.81      0.69      0.75      7296
           1       0.94      0.97      0.96     39181

    accuracy                           0.93     46477
   macro avg       0.88      0.83      0.85     46477
weighted avg       0.92      0.93      0.92     46477



Classe prédite,0,1
Classe réelle,,
0,5036,2260
1,1177,38004


#### XGBoost

In [12]:
# Créer un classificateur XGBoost.
xgb_clf = xgb.XGBClassifier()

# Entraîner le classificateur sur l'ensemble d'entraînement.
xgb_clf.fit(X_train_concat, y_train)
joblib.dump(xgb_clf, 'xgb_clf.joblib')

# Faire des prédictions sur l'ensemble de test.
predictions = xgb_clf.predict(X_test_concat)

In [13]:
# Affichage du rapport de classification du modèle XGBoost.
print("Rapport de classification pour XGBoost :\n", classification_report(y_test, predictions))

# Calcul et affichage de la matrice de confusion pour XGBoost.
confusion_matrix = pd.crosstab(y_test, predictions, rownames=['Classe réelle'], colnames=['Classe prédite pour XGBoost'])
confusion_matrix

Rapport de classification pour XGBoost :
               precision    recall  f1-score   support

           0       0.82      0.80      0.81      7296
           1       0.96      0.97      0.97     39181

    accuracy                           0.94     46477
   macro avg       0.89      0.88      0.89     46477
weighted avg       0.94      0.94      0.94     46477



Classe prédite pour XGBoost,0,1
Classe réelle,,
0,5838,1458
1,1256,37925


#### Random Forest

In [14]:
# Créer un classificateur Random Forest.
rf_clf = RandomForestClassifier()

# Entraîner le classificateur Random Forest sur l'ensemble d'entraînement.
rf_clf.fit(X_train_concat, y_train)
joblib.dump(rf_clf, 'rf_clf.joblib')

# Faire des prédictions sur l'ensemble de test avec Random Forest.
predictions = rf_clf.predict(X_test_concat)

In [15]:
# Affichage du rapport de classification du modèle Random Forest.
print("Rapport de classification pour Random Forest :\n", classification_report(y_test, predictions))

# Calcul et affichage de la matrice de confusion pour Random Forest.
confusion_matrix = pd.crosstab(y_test, predictions, rownames=['Classe réelle'], colnames=['Classe prédite pour Random Forest'])
confusion_matrix

Rapport de classification pour Random Forest :
               precision    recall  f1-score   support

           0       0.86      0.69      0.77      7296
           1       0.95      0.98      0.96     39181

    accuracy                           0.93     46477
   macro avg       0.90      0.84      0.87     46477
weighted avg       0.93      0.93      0.93     46477



Classe prédite pour Random Forest,0,1
Classe réelle,,
0,5069,2227
1,810,38371


#### Logistic Regression

In [16]:
# Créer et entraîner un classificateur Logistic Regression.
lr_clf = LogisticRegression()

# Entraîner le classificateur Logistic Regression sur l'ensemble d'entraînement.
lr_clf.fit(X_train_concat, y_train)
joblib.dump(lr_clf, 'lr_clf.joblib')

# Faire des prédictions sur l'ensemble de test avec Logistic Regression.
predictions = lr_clf.predict(X_test_concat)

c:\Users\Windows\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
# Affichage du rapport de classification du modèle Logistic Regression.
print("Rapport de classification pour Logistic Regression :\n", classification_report(y_test, predictions))

# Calcul et affichage de la matrice de confusion pour Logistic Regression.
confusion_matrix = pd.crosstab(y_test, predictions, rownames=['Classe réelle'], colnames=['Classe prédite pour Logistic Regression'])
confusion_matrix

Rapport de classification pour Logistic Regression :
               precision    recall  f1-score   support

           0       0.79      0.67      0.72      7296
           1       0.94      0.97      0.95     39181

    accuracy                           0.92     46477
   macro avg       0.86      0.82      0.84     46477
weighted avg       0.92      0.92      0.92     46477



Classe prédite pour Logistic Regression,0,1
Classe réelle,,
0,4892,2404
1,1335,37846


#### Support Vector Machine

In [18]:
# Créer et entraîner un classificateur Support Vector Machine.
svm_clf = SVC()

# Entraîner le classificateur Support Vector Machine sur l'ensemble d'entraînement.
svm_clf.fit(X_train_concat, y_train)
joblib.dump(svm_clf, 'svm_clf.joblib')

# Faire des prédictions sur l'ensemble de test avec Support Vector Machine.
svm_predictions = svm_clf.predict(X_test_concat)

In [19]:
# Affichage du rapport de classification du modèle Support Vector Machine.
print("Rapport de classification pour Support Vector Machine :\n", classification_report(y_test, predictions))

# Calcul et affichage de la matrice de confusion pour Support Vector Machine.
confusion_matrix = pd.crosstab(y_test, predictions, rownames=['Classe réelle'], colnames=['Classe prédite pour Support Vector Machine'])
confusion_matrix

Rapport de classification pour Support Vector Machine :
               precision    recall  f1-score   support

           0       0.79      0.67      0.72      7296
           1       0.94      0.97      0.95     39181

    accuracy                           0.92     46477
   macro avg       0.86      0.82      0.84     46477
weighted avg       0.92      0.92      0.92     46477



Classe prédite pour Support Vector Machine,0,1
Classe réelle,,
0,4892,2404
1,1335,37846


#### Naive Bayes

In [20]:
# Créer un classificateur clf en utilisant la méthode MultinomialNB.
nb_clf = MultinomialNB()

# Entraîner le classificateur Naive Bayes sur l'ensemble d'entraînement.
nb_clf.fit(X_train_commentaire_vectorized, y_train)
joblib.dump(nb_clf, 'nb_clf.joblib')

# Faire des prédictions sur l'ensemble de test avec Naive Bayes.
predictions = nb_clf.predict(X_test_commentaire_vectorized)

In [21]:
# Affichage du rapport de classification du modèle Naive Bayes.
print("Rapport de classification pour Naive Bayes :\n", classification_report(y_test, predictions))

# Calcul et affichage de la matrice de confusion pour Naive Bayes.
confusion_matrix = pd.crosstab(y_test, predictions, rownames=['Classe réelle'], colnames=['Classe prédite pour Naive Bayes'])
confusion_matrix

Rapport de classification pour Naive Bayes :
               precision    recall  f1-score   support

           0       0.72      0.86      0.79      7296
           1       0.97      0.94      0.96     39181

    accuracy                           0.93     46477
   macro avg       0.85      0.90      0.87     46477
weighted avg       0.93      0.93      0.93     46477



Classe prédite pour Naive Bayes,0,1
Classe réelle,,
0,6285,1011
1,2410,36771
